# Open RAG

In [17]:
import os
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

In [18]:
# Configurar la clave de API de OpenAI
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_KEY")

# Cargar el archivo de texto
loader = TextLoader("movies.txt", encoding="utf-8")
documents = loader.load()

# Dividir el texto en fragmentos
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
docs = text_splitter.split_documents(documents)

# Initialize embeddings (e.g., OpenAI)
embeddings = OpenAIEmbeddings(
    openai_api_key=os.getenv("OPENAI_KEY"),
    model="text-embedding-3-small"
)

# Store embeddings in vector database (Chroma in-memory DB)
vectorstore = Chroma.from_documents(docs, embeddings)

In [19]:
# Recuperar los primeros 4 documentos almacenados
resultados = vectorstore.get(limit=6, include=["embeddings", "documents", "metadatas"])

# Iterar sobre los documentos y mostrarlos
for i, doc in enumerate(resultados["documents"]):
    print(f"Documento {i+1}:")
    print(doc)
    print("-" * 50)

# Iterar sobre los documentos y mostrarlos
for i, doc in enumerate(resultados["embeddings"]):
    print(f"Documento {i+1}:")
    print(doc)
    print("-" * 50)

Documento 1:
Title: The Shawshank Redemption (1994)
Movie Code: MOV-001
--------------------------------------------------
Documento 2:
Description:
--------------------------------------------------
Documento 3:
Adapted from Stephen King’s novella, Rita Hayworth and Shawshank Redemption, this film chronicles
--------------------------------------------------
Documento 4:
film chronicles the life of Andy Dufresne, a banker wrongly convicted of murdering his wife and her
--------------------------------------------------
Documento 5:
his wife and her lover. Andy finds himself in the brutal environment of Shawshank Prison, where he
--------------------------------------------------
Documento 6:
Prison, where he befriends fellow inmate Red. Over two decades, Andy faces despair, abuse, and
--------------------------------------------------
Documento 1:
[-0.04190804  0.02641674 -0.0229574  ... -0.02288752  0.02277104
 -0.02034834]
--------------------------------------------------
Documento

In [3]:
# Configurar el modelo de lenguaje
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# Crear la cadena de preguntas y respuestas
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=vectorstore.as_retriever())

In [4]:
# Realizar una consulta
query = "¿Que peliculas conoces?"
result = qa_chain.invoke(query)

print("Pregunta:", query)
print("Respuesta:", result)

Pregunta: ¿Que peliculas conoces?
Respuesta: {'query': '¿Que peliculas conoces?', 'result': 'Conozco las películas "Inception" (2010) y "The Godfather" (1972).'}


In [9]:
# Realizar una consulta
query = "¿Conoces la pelicula Parasite? Si es asi dame detalles de ella."
result = qa_chain.invoke(query)

print("Pregunta:", query)
print("Respuesta:", result["result"])

Pregunta: ¿Conoces la pelicula El padrino? Si es asi dame detalles de ella.
Respuesta: Sí, conozco la película "El Padrino" (título original: "The Godfather"), que fue lanzada en 1972. Es un épico drama criminal dirigido por Francis Ford Coppola y basado en la novela de Mario Puzo. La película narra la historia de la familia criminal Corleone, centrándose en el patriarca Vito Corleone, interpretado por Marlon Brando, y su hijo Michael Corleone, interpretado por Al Pacino. Es una obra icónica que explora temas de poder, lealtad y traición dentro del mundo del crimen organizado.


In [8]:
# Realizar una consulta
query = "¿Cuándo se estrenó la primera película de Star Wars?"
result = qa_chain.invoke(query)

print("Pregunta:", query)
print("Respuesta:", result["result"])

Pregunta: ¿Cuándo se estrenó la primera película de Star Wars?
Respuesta: La primera película de Star Wars, conocida como "Star Wars: Episode IV - A New Hope", se estrenó el 25 de mayo de 1977.
